In [1]:
import spacy
import pandas as pd
import numpy as np
from glob import glob
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import datasets
from sklearn import metrics
from sklearn import model_selection
import sys

# Display plots in this notebook, instead of externally. 
from pylab import rcParams
rcParams['figure.figsize'] = 16, 8
%matplotlib inline

import ete3 
import seaborn
import collections
import math
import os
import random
import zipfile
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
import gensim
from sklearn.manifold import TSNE
import matplotlib
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file

In [2]:
features_test = pd.read_csv('./dengue_features_test.csv', index_col=0, parse_dates=True, delimiter=',')
features_train = pd.read_csv('./dengue_features_train.csv', index_col=[0,1,2], parse_dates=True, delimiter=',')
labels_train = pd.read_csv('./dengue_labels_train.csv', index_col=[0,1,2], parse_dates=True, delimiter=',')
#sformat = pd.read_csv('./submission_format.csv', index_col=0, parse_dates=True, delimiter=',')

In [3]:
features_test.head(10)

,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
city,,,,,,,,,,,,,,,,,,,,,
sj,2008,18,2008-04-29,-0.018900,-0.018900,0.102729,0.091200,78.60,298.492857,298.550000,...,25.37,78.781429,78.60,15.918571,3.128571,26.528571,7.057143,33.3,21.7,75.2
sj,2008,19,2008-05-06,-0.018000,-0.012400,0.082043,0.072314,12.56,298.475714,298.557143,...,21.83,78.230000,12.56,15.791429,2.571429,26.071429,5.557143,30.0,22.2,34.3
sj,2008,20,2008-05-13,-0.001500,NaN,0.151083,0.091529,3.66,299.455714,299.357143,...,4.12,78.270000,3.66,16.674286,4.428571,27.928571,7.785714,32.8,22.8,3.0
sj,2008,21,2008-05-20,NaN,-0.019867,0.124329,0.125686,0.00,299.690000,299.728571,...,2.20,73.015714,0.00,15.775714,4.342857,28.057143,6.271429,33.3,24.4,0.3
sj,2008,22,2008-05-27,0.056800,0.039833,0.062267,0.075914,0.76,299.780000,299.671429,...,4.36,74.084286,0.76,16.137143,3.542857,27.614286,7.085714,33.3,23.3,84.1
sj,2008,23,2008-06-03,-0.044000,-0.030467,0.132000,0.083529,71.17,299.768571,299.728571,...,22.55,76.557143,71.17,16.667143,2.857143,28.000000,5.171429,32.8,25.0,27.7
sj,2008,24,2008-06-10,-0.044300,-0.024925,0.132271,0.159157,48.99,300.062857,300.007143,...,13.10,76.844286,48.99,17.010000,3.157143,27.400000,6.042857,31.1,23.3,91.7
sj,2008,25,2008-06-17,NaN,0.082150,0.144371,0.116729,30.81,300.484286,300.578571,...,7.20,76.870000,30.81,17.420000,3.900000,28.757143,6.985714,34.4,24.4,0.3
sj,2008,26,2008-06-24,0.010800,0.049900,0.100571,0.117329,8.02,300.601429,300.621429,...,17.10,77.395714,8.02,17.678571,2.785714,28.657143,6.242857,32.8,23.9,28.7


In [4]:
# Seperate data for San Juan
sj_train_features = features_train.loc['sj']
sj_train_labels = labels_train.loc['sj']
sj_test = features_test.loc['sj']

# Separate data for Iquitos
iq_train_features = features_train.loc['iq']
iq_train_labels = labels_train.loc['iq']
iq_test = features_test.loc['iq']

In [5]:
sj_train_features.head(10)

week_start_date   ndvi_ne   ndvi_nw   ndvi_se   ndvi_sw  \
year       weekofyear                                                           
1990-01-01 18              1990-04-30  0.122600  0.103725  0.198483  0.177617   
           19              1990-05-07  0.169900  0.142175  0.162357  0.155486   
           20              1990-05-14  0.032250  0.172967  0.157200  0.170843   
           21              1990-05-21  0.128633  0.245067  0.227557  0.235886   
           22              1990-05-28  0.196200  0.262200  0.251200  0.247340   
           23              1990-06-04       NaN  0.174850  0.254314  0.181743   
           24              1990-06-11  0.112900  0.092800  0.205071  0.210271   
           25              1990-06-18  0.072500  0.072500  0.151471  0.133029   
           26              1990-06-25  0.102450  0.146175  0.125571  0.123600   
           27              1990-07-02       NaN  0.121550  0.160683  0.202567   

                       precipitation_amt_mm  reanalysis_air_temp_k  \
year       weekofyear                                                
1990-01-01 18                         12.42             297.572857   
           19                         22.82             298.211429   
           20                         34.54             298.781429   
           21                         15.36             298.987143   
           22                          7.52             299.518571   
           23                          9.58             299.630000   
           24                          3.48             299.207143   
           25                        151.12             299.591429   
           26                         19.32             299.578571   
           27                         14.41             300.154286   

                       reanalysis_avg_temp_k  reanalysis_dew_point_temp_k  \
year       weekofyear                                                       
1990-01-01 18                     297.742857                   292.414286   
           19                     298.442857                   293.951429   
           20                     298.878571                   295.434286   
           21                     299.228571                   295.310000   
           22                     299.664286                   295.821429   
           23                     299.764286                   295.851429   
           24                     299.221429                   295.865714   
           25                     299.528571                   296.531429   
           26                     299.557143                   296.378571   
           27                     300.278571                   296.651429   

                       reanalysis_max_air_temp_k        ...          \
year       weekofyear                                   ...           
1990-01-01 18                              299.8        ...           
           19                              300.9        ...           
           20                              300.5        ...           
           21                              301.4        ...           
           22                              301.9        ...           
           23                              302.4        ...           
           24                              301.3        ...           
           25                              300.6        ...           
           26                              302.1        ...           
           27                              302.3        ...           

                       reanalysis_precip_amt_kg_per_m2  \
year       weekofyear                                    
1990-01-01 18                                    32.00   
           19                                    17.94   
           20                                    26.10   
           21                                    13.90   
           22                                    12.20   
           23           

In [6]:
#mapping = {'set': 0, 'test': 1}

#In [10]: df.replace({'set': mapping, 'tesst': mapping})

In [7]:
print('San Juan')
print('features: ', sj_train_features.shape)
print('labels  : ', sj_train_labels.shape)

print('\nIquitos')
print('features: ', iq_train_features.shape)
print('labels  : ', iq_train_labels.shape)

San Juan
features:  (936, 21)
labels  :  (936, 1)

Iquitos
features:  (520, 21)
labels  :  (520, 1)


In [8]:
# Remove `week_start_date` string.
sj_train_features.drop('week_start_date', axis=1, inplace=True)
iq_train_features.drop('week_start_date', axis=1, inplace=True)

/home/brad/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/brad/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
sj_train_features.head(5)

ndvi_ne   ndvi_nw   ndvi_se   ndvi_sw  \
year       weekofyear                                           
1990-01-01 18          0.122600  0.103725  0.198483  0.177617   
           19          0.169900  0.142175  0.162357  0.155486   
           20          0.032250  0.172967  0.157200  0.170843   
           21          0.128633  0.245067  0.227557  0.235886   
           22          0.196200  0.262200  0.251200  0.247340   

                       precipitation_amt_mm  reanalysis_air_temp_k  \
year       weekofyear                                                
1990-01-01 18                         12.42             297.572857   
           19                         22.82             298.211429   
           20                         34.54             298.781429   
           21                         15.36             298.987143   
           22                          7.52             299.518571   

                       reanalysis_avg_temp_k  reanalysis_dew_point_temp_k  \
year       weekofyear                                                       
1990-01-01 18                     297.742857                   292.414286   
           19                     298.442857                   293.951429   
           20                     298.878571                   295.434286   
           21                     299.228571                   295.310000   
           22                     299.664286                   295.821429   

                       reanalysis_max_air_temp_k  reanalysis_min_air_temp_k  \
year       weekofyear                                                         
1990-01-01 18                              299.8                      295.9   
           19                              300.9                      296.4   
           20                              300.5                      297.3   
           21                              301.4                      297.0   
           22                              301.9                      297.5   

                       reanalysis_precip_amt_kg_per_m2  \
year       weekofyear                                    
1990-01-01 18                                    32.00   
           19                                    17.94   
           20                                    26.10   
           21                                    13.90   
           22                                    12.20   

                       reanalysis_relative_humidity_percent  \
year       weekofyear                                         
1990-01-01 18                                     73.365714   
           19                                     77.368571   
           20                                     82.052857   
           21                                     80.337143   
           22                                     80.460000   

                       reanalysis_sat_precip_amt_mm  \
year       weekofyear                                 
1990-01-01 18                                 12.42   
           19                                 22.82   
           20                                 34.54   
           21                                 15.36   
           22                                  7.52   

                       reanalysis_specific_humidity_g_per_kg  \
year       weekofyear                                          
1990-01-01 18                                      14.012857   
           19                                      15.372857   
           20                                      16.848571   
           21                                      16.672857   
           22                                      17.210000   

                       reanalysis_tdtr_k  station_avg_temp_c  \
year       weekofyear                                          
1990-01-01 18                   2.628571           25.442857   
           19                   2.371429           26.714286   
           20                   2.300000   

In [10]:
# Null check
pd.isnull(sj_train_features).any()

ndvi_ne                                  True
ndvi_nw                                  True
ndvi_se                                  True
ndvi_sw                                  True
precipitation_amt_mm                     True
reanalysis_air_temp_k                    True
reanalysis_avg_temp_k                    True
reanalysis_dew_point_temp_k              True
reanalysis_max_air_temp_k                True
reanalysis_min_air_temp_k                True
reanalysis_precip_amt_kg_per_m2          True
reanalysis_relative_humidity_percent     True
reanalysis_sat_precip_amt_mm             True
reanalysis_specific_humidity_g_per_kg    True
reanalysis_tdtr_k                        True
station_avg_temp_c                       True
station_diur_temp_rng_c                  True
station_max_temp_c                       True
station_min_temp_c                       True
station_precip_mm                        True
dtype: bool

In [11]:
sj_train_features.fillna(method='ffill', inplace=True)
iq_train_features.fillna(method='ffill', inplace=True)

/home/brad/.local/lib/python3.5/site-packages/pandas/core/frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [12]:
print('San Juan')
print('mean: ', sj_train_labels.mean()[0])
print('var :', sj_train_labels.var()[0])

print('\nIquitos')
print('mean: ', iq_train_labels.mean()[0])
print('var :', iq_train_labels.var()[0])

San Juan
mean:  34.1805555556
var : 2640.04543969

Iquitos
mean:  7.56538461538
var : 115.895523937


In [13]:
sj_train_features.head(10)

ndvi_ne   ndvi_nw   ndvi_se   ndvi_sw  \
year       weekofyear                                           
1990-01-01 18          0.122600  0.103725  0.198483  0.177617   
           19          0.169900  0.142175  0.162357  0.155486   
           20          0.032250  0.172967  0.157200  0.170843   
           21          0.128633  0.245067  0.227557  0.235886   
           22          0.196200  0.262200  0.251200  0.247340   
           23          0.196200  0.174850  0.254314  0.181743   
           24          0.112900  0.092800  0.205071  0.210271   
           25          0.072500  0.072500  0.151471  0.133029   
           26          0.102450  0.146175  0.125571  0.123600   
           27          0.102450  0.121550  0.160683  0.202567   

                       precipitation_amt_mm  reanalysis_air_temp_k  \
year       weekofyear                                                
1990-01-01 18                         12.42             297.572857   
           19                         22.82             298.211429   
           20                         34.54             298.781429   
           21                         15.36             298.987143   
           22                          7.52             299.518571   
           23                          9.58             299.630000   
           24                          3.48             299.207143   
           25                        151.12             299.591429   
           26                         19.32             299.578571   
           27                         14.41             300.154286   

                       reanalysis_avg_temp_k  reanalysis_dew_point_temp_k  \
year       weekofyear                                                       
1990-01-01 18                     297.742857                   292.414286   
           19                     298.442857                   293.951429   
           20                     298.878571                   295.434286   
           21                     299.228571                   295.310000   
           22                     299.664286                   295.821429   
           23                     299.764286                   295.851429   
           24                     299.221429                   295.865714   
           25                     299.528571                   296.531429   
           26                     299.557143                   296.378571   
           27                     300.278571                   296.651429   

                       reanalysis_max_air_temp_k  reanalysis_min_air_temp_k  \
year       weekofyear                                                         
1990-01-01 18                              299.8                      295.9   
           19                              300.9                      296.4   
           20                              300.5                      297.3   
           21                              301.4                      297.0   
           22                              301.9                      297.5   
           23                              302.4                      298.1   
           24                              301.3                      297.7   
           25                              300.6                      298.4   
           26                              302.1                      297.7   
           27                              302.3                      298.7   

                       reanalysis_precip_amt_kg_per_m2  \
year       weekofyear                                    
1990-01-01 18                                    32.00   
           19                                    17.94   
           20                                    26.10   
           21                                    13.90   
           22                                    12.20   
           23                                    26.49   
           24                                    38.60   
      

In [14]:
# Build 3 layer DNN with 10, 20, 10 units respectively.
feature_columns_sj = tf.contrib.learn.infer_real_valued_columns_from_input(sj_train_features)
feature_columns_iq = tf.contrib.learn.infer_real_valued_columns_from_input(iq_train_features)


regressor_sj = tf.contrib.learn.DNNRegressor(feature_columns=feature_columns_sj, hidden_units=[10, 20, 10])
regressor_iq = tf.contrib.learn.DNNRegressor(feature_columns=feature_columns_iq, hidden_units=[10, 20, 10])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbbf060f550>, '_session_config': None, '_task_type': None, '_save_summary_steps': 100, '_task_id': 0, '_tf_random_seed': None, '_num_worker_replicas': 0, '_evaluation_master': '', '_master': '', '_num_ps_replicas': 0, '_is_chief': True, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmpcds37jvw', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_environment': 'local'}
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbbf060f860>, '_session_config': None, '_task_type': None, '_save_summary_steps': 100, '_task_id': 0, '_tf_random_seed': None, '_num_worker_replicas': 0, '_evaluation_m

In [16]:
# Fit and predict.

#regressor_sj.fit(sj_train_features, steps=50)

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


ValueError: Missing training_op.

In [ ]:
regressor_iq.fit(iq_train_features, steps=2)

In [ ]:
predictions = list(regressor_sj.predict(sj_test, as_iterable=True))
predictions = list(regressor_iq.predict(iq_test, as_iterable=True))

score1 = metrics.accuracy_score(sj_test, predictions)
score2 = metrics.accuracy_score(iq_test, predictions)

print('Accuracy: {0:f}'.format(score))